# Lecture2 Word Vectors

### 如何在计算机中使用词意

通用解决方案：使用例如WordNet,一个包含列表的**同义词集**和**上位词集**

In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
from nltk.corpus import wordnet as wn
for synset in wn.synsets("good"):
    print("(%s)" % synset.pos())
    print(", ".join([l.name() for l in synset.lemmas()]))

(n)
good
(n)
good, goodness
(n)
good, goodness
(n)
commodity, trade_good, good
(a)
good
(s)
full, good
(a)
good
(s)
estimable, good, honorable, respectable
(s)
beneficial, good
(s)
good
(s)
good, just, upright
(s)
adept, expert, good, practiced, proficient, skillful, skilful
(s)
good
(s)
dear, good, near
(s)
dependable, good, safe, secure
(s)
good, right, ripe
(s)
good, well
(s)
effective, good, in_effect, in_force
(s)
good
(s)
good, serious
(s)
good, sound
(s)
good, salutary
(s)
good, honest
(s)
good, undecomposed, unspoiled, unspoilt
(s)
good
(r)
well, good
(r)
thoroughly, soundly, good


In [3]:
from nltk.corpus import wordnet as wn
panda = wn.synsets("panda.n.01")
hyper = lambda s: s.hypernyms()
list(panda.closure(hyper))

AttributeError: 'list' object has no attribute 'closure'

### WordNet存在的问题

- 作为资源很好，但缺乏细微差别

	- “proficient”与“good”是同义词，但这只在某些情况下是正确的

- 缺少词语的新含义

	- 例如 wicked, badass, nifty, wizard, genius, ninja, bombest
	- 不可能持续更新

- 主观
- 需要人工创建和适应
- 很难计算词语的相似性

### 将单词表示为离散符号

在传统的NLP中，我们将单词视为离散的符号

词语可以被表示为one-hot向量（向量中有一维是1，其余为0），向量维数即为词典中词语个数

### 将单词作为离散符号的问题

在网络搜索中，如果用户搜索“Seattle motel”，我们希望检索结果包含““Seattle hotel”

但：

> motel = [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
> hotel = [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]

这两个向量是正交的，one-hot向量没有自然的**相似性**概念，所以我们要尝试在向量本身中编码相似性

### 根据上下文表示单词

- 核心思想：**一个词的含义是由经常出现在这个词附近的词给出的**

- 当一个词出现在文章中时，其上下文就是出现在它附近的词（在一定范围内）

- 使用w的许多上下文来构建w的表示

    ![JPEG 图像-9FDC3ECCBDAB-1.jpeg](https://i.loli.net/2018/04/09/5acacb2c8585d.jpeg)
    
### 词向量

为每一个单词建立一个密集的向量选择与其出现在类似语境下的向量

Note: word vectors are sometimes called word embeddings or word representations.

### Word2vec：概述

Word2vec（Mikolov et al。2013）是一个词向量的学习框架

理念：
- 我们有一个大的文本语料库
- 固定词汇表中的每个词都由一个向量表示
- 通过文本中的每个位置t，它有一个中心词c和上下文（“外部”）字o
- 使用c和o的单词向量的相似度来计算给定c的概率（反之亦然）
- 继续调整单词向量以最大化这个概率

计算$P(w_{t+j}|w_{t})$

![屏幕快照 2018-04-09 上午10.13.26.png](https://i.loli.net/2018/04/09/5acacc784f30d.png)

![屏幕快照 2018-04-09 上午10.47.55.png](https://i.loli.net/2018/04/09/5acad471f35bf.png)

### Word2vec：目标函数

对于每一个位置t=1,...,T，，预测固定大小为m的窗口内的上下文单词，得到中心词$w_{j}$。

![屏幕快照 2018-04-09 上午10.49.25.png](https://i.loli.net/2018/04/09/5acad4c97f473.png)

目标函数（有时称为成本函数损失函数）⌡(θ)是（平均）负对数似然性：

![屏幕快照 2018-04-09 上午11.00.16.png](https://i.loli.net/2018/04/09/5acad7501c623.png)

最小化目标函数⟺最大化预测准确性

我们想要最小化目标函数⌡(θ)，为了计算$P(w_{t+j}|w_{t};θ)$，我们将用两个向量表示一个词w：

- 当w是中心词时用$v_{w}$
- 当w是上下文词时用$u_{w}$

然后对于中心词c和上下文词o：

![屏幕快照 2018-04-09 上午11.06.28.png](https://i.loli.net/2018/04/09/5acad8c14e526.png)

### Word2vec：预测功能
![屏幕快照 2018-04-09 下午3.17.25.png](https://i.loli.net/2018/04/09/5acb139010a77.png)

点积比较o和c的相似性。较大的点积=较大的概率

取指数后，对整个词汇表进行归一化

Softmax function：从实数空间到概率分布的标准映射方法

![屏幕快照 2018-04-09 下午3.20.03.png](https://i.loli.net/2018/04/09/5acb1432365d7.png)

softmax函数将任意值$x_{i}$映射到概率分布$p_{i}$，在深度学习中使用很频繁

指数函数可以把实数映射成正数，然后归一化得到概率。softmax之所叫softmax，是因为指数函数会导致较大的数变得更大，小数变得微不足道；这种选择作用类似于max函数。

### 训练模型：计算所有矢量梯度

把所有参数写进向量θ，对d维的词向量和大小V的词表来讲，有：

![屏幕快照 2018-04-09 下午3.38.08.png](https://i.loli.net/2018/04/09/5acb18673af45.png)

每个单词都有两个向量

然后我们优化这些参数

### 梯度的推导

具体推导方法见[word2vec原理推导与代码分析](http://www.hankcs.com/nlp/word2vec.html#h3-5)

### 损失/目标函数

梯度有了，参数减去梯度就能朝着最小值走了

![屏幕快照 2018-04-09 下午5.36.23.png](https://i.loli.net/2018/04/09/5acb3421da2b2.png)

只有一句比较新鲜，神经网络喜欢嘈杂的算法，这可能是随机梯度下降(SGD)成功的另一原因
